In [1]:
#from screening import experiment_get_ellipsoid
from tools import find_best_svm, compute_ellipsoid_volume, find_best_lasso, scoring_classif
from experiment import experiment_get_ellipsoid
from loaders import load_experiment
from screening import rank_dataset_accelerated, compute_subgradient
from safelog import SafeLogistic
import numpy as np
from sklearn.utils import shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
path = './datasets/'

In [2]:
def update_ellipsoid(z, A, g):
    p = z.size
    A_g = A.dot(g)
    den = np.sqrt(g.dot(A_g))
    g = (1 / den) * g
    A_g = A.dot(g)
    z = z - (1 / (p + 1)) * A_g
    A = (p ** 2 / (p ** 2 - 1)) * (A - (2 / (p + 1)) * np.outer(A_g, A_g))
    return z, A

def iterate_ellipsoids(D, y, z_init, r_init, lmbda, mu, loss, penalty, n_ellipsoid_steps, intercept):
    k = 0
    z = z_init
    A = r_init * np.identity(D.shape[1])
    while k < n_ellipsoid_steps:
        g = compute_subgradient(z, D, y, lmbda, mu, loss, penalty, intercept)
        z, A = update_ellipsoid(z, A, g)
        k += 1 
    return z, A

def experiment_get_ellipsoid_old(X, y, intercept, better_init, better_radius, loss, penalty, 
                                lmbda, classification, mu, n_ellipsoid_steps):
    if intercept:
        z_init = np.zeros(X.shape[1] + 1)
        r_init = X.shape[1] + 1
    else:
        z_init = np.zeros(X.shape[1])
        r_init = X.shape[1]
    if better_init != 0:
        est = fit_estimator(X, y, loss, penalty, lmbda, intercept, max_iter=better_init)
        if classification:
            z_init = est.coef_[0]
            if intercept:
                z_init = np.append(z_init, est.intercept_)
        else:
            z_init = est.coef_
            if intercept:
                z_init = np.append(z_init, est.intercept_)
    if better_radius != 0:
        r_init = float(better_radius)
    
    z, A = iterate_ellipsoids(X, y, z_init, r_init, lmbda, mu, loss, penalty, n_ellipsoid_steps, intercept)
    return z, A

# OBJECTIF

- Tester la méthode ellipsoide en petite (Higgs), moyenne (MNIST) et grande dimension (20newsgroups) pour voir la vitesse de convergence vers la solution et le rayon de l'ellipsoide.
- En particulier, regarder combien d'étapes sont nécessaires. 
- Comment améliorer ? Est-ce que quelques pas de SGD aident?

In [4]:
X, y = load_experiment(dataset='mnist', synth_params=None, size=100, redundant=0, noise=0, classification=True, path=path)

Shape of original MNIST (60000, 2304)


In [4]:
X.shape

(100, 2304)

In [5]:
svm, score = find_best_svm(X, y)
print(svm, score)
print(svm.coef_, np.array(svm.coef_[0]).reshape(1,-1))

KeyboardInterrupt: 

In [ ]:
svm_sgd = SGDClassifier()
svm_sgd.fit(X,y)
print(svm_sgd.coef_)

In [41]:
print(svm.coef_, svm.intercept_)
if svm.intercept_ == 0:
    ground_truth = svm.coef_[0]
else:
    ground_truth = np.concatenate((svm.coef_, svm.intercept_), axis=None)
print('NO GO-BELOW ZONE FOR THE RADIUS', np.linalg.norm(ground_truth)) # check that it is inside the initial ball
print('RADIUS', X.shape[1])
print(len(np.nonzero(ground_truth)[0]))

[[-0.02213154  0.00208679  0.         ...,  0.         -0.01719506  0.        ]] 0.0
NO GO-BELOW ZONE FOR THE RADIUS 1.28825700512
RADIUS 10094
4097


In [38]:
for n_ellipsoid_steps in [10, 100, 1000, 10000]:
    z, scaling, L, I_k_vec, _ = experiment_get_ellipsoid(X, y, intercept=True, better_init=0, better_radius=X.shape[1], 
                                                         loss='hinge', penalty='l2', lmbda=1/100, classification=True, mu=1, n_ellipsoid_steps=n_ellipsoid_steps)
    #print('L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT', np.linalg.norm(z - ground_truth) / np.linalg.norm(ground_truth))
    #print('GROUND TRUTH', ground_truth, 'ELLIPSOID OUTPUT', z)
    M = L.dot(np.dot(np.diag(I_k_vec), L.T))
    A = scaling * np.identity(X.shape[1]+1) - M
    radius = np.linalg.eig(A)[0]
    #radius = np.max(radius)
    print('RADIUS', np.sort(radius))
    print('SCALING', scaling)
    #print('I_k_vec', I_k_vec)
    radius_ = np.linalg.eig(M)[0]
    print('RADIUS_', np.sort(radius_))
    #volume = compute_ellipsoid_volume(radius)
    #print('VOLUME', volume)

Time to compute z, A and g: 0.004828453063964844


ValueError: too many values to unpack (expected 5)

# CLIPPING DE L'ELLIPSOIDE

In [49]:
a = np.array([1,2,5,7])
np.clip(a,0,5)

array([1, 2, 5, 5])

# Safe Logistic

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
safelog = SafeLogistic(mu=0, lmbda=0.0, max_iter=10000)
safelog.fit(X_train, y_train)
ground_truth = safelog.coef_[0]
print(ground_truth)
print(scoring_classif(safelog, X_test, y_test))

[ 1.12239618  1.04578339  1.65307495 ...,  0.2721837   0.80306286
  0.75354699]
0.9


In [6]:
mu=0
lmbda = 0.0
intercept=False 
classification=True
loss='safe_logistic'
penalty='l2'
cut=True
better_init=10
better_radius=10 #X.shape[1]

In [7]:
for n_ellipsoid_steps in [1, 10, 100, 1000, 10000]: #, 100]: #, 1000]: #, 100000]:
    z, scaling, L, I_k_vec, g , _= experiment_get_ellipsoid(X, y, intercept=intercept, better_init=better_init, better_radius=better_radius, 
                                                         loss=loss, penalty=penalty, lmbda=lmbda, 
                                                         classification=classification, mu=mu, 
                                                         n_ellipsoid_steps=n_ellipsoid_steps)
    #z_old, A_old = experiment_get_ellipsoid_old(X, y, intercept=intercept, better_init=better_init, better_radius=better_radius, 
                                                         #loss=loss, penalty=penalty, lmbda=lmbda, 
                                                         #classification=classification, mu=mu, 
                                                         #n_ellipsoid_steps=n_ellipsoid_steps)
    print('L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT', np.linalg.norm(z - ground_truth) / np.linalg.norm(ground_truth))
    #print('L2 ... WITH OLD METHOD', np.linalg.norm(z_old - ground_truth) / np.linalg.norm(ground_truth))
    #print('GROUND TRUTH', ground_truth, 'ELLIPSOID OUTPUT', z)
    
    ranking = rank_dataset_accelerated(X, y, z, scaling, L, I_k_vec, g, lmbda, mu, classification, 
    loss, penalty, intercept, cut)
    print('RANKING', ranking, np.argsort(ranking))
    M = L.dot(np.dot(np.diag(I_k_vec), L.T))
    A = scaling * np.identity(X.shape[1]) - M
    radius, _ = np.linalg.eigh(A)
    #print(radius)
    #print(A)
    #radius = np.linalg.eig(A)[0]
    #print(radius)
    #radius = np.max(radius)
    #print('RADIUS', np.sort(radius))
    #print('SCALING', scaling)
    #print('I_k_vec', I_k_vec)
    #radius_ = np.linalg.eig(M)[0]
    #print('RADIUS_', np.sort(radius_))
    #volume = compute_ellipsoid_volume(radius)
    #print('VOLUME', volume)

Time to compute z, A and g: 0.0020401477813720703
L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT 0.603403975718
Time to rank the entire dataset: 0.014620780944824219
RANKING [ -4.54185901e-01  -3.38939730e-01   2.69609573e-01  -1.23495394e-01
   2.26549916e+00  -3.42501453e-01   1.85453068e+00  -7.06207936e-02
   1.75835383e+00  -1.93800484e-01   1.38394695e+00  -5.05616130e-01
  -2.13222892e-01   3.25237495e-01   1.55315930e+00   1.57705610e+00
  -8.82440601e-01   1.05410761e+00   1.14735270e+00   2.09864894e+00
   6.33057310e-01   7.15459996e-02   2.26173274e+00  -4.57271040e-02
  -1.63068900e+00  -9.81122915e-01   2.28316965e+00  -1.79580240e-01
  -8.07068946e-01   1.25159918e+00   7.84508426e-01  -4.16965495e-01
   6.52249844e-01   2.70792314e+00   5.82398228e-01  -5.01220088e-01
   3.38333532e-01  -4.03962610e-01  -7.13702383e-01   1.00595310e+00
   7.03819828e-01   1.36674359e+00   1.41592282e+00   1.47134852e+00
   1.65527095e+00   2.07036279e+00   2.02113214e+00  -6.8473

# Lasso 

In [46]:
lasso, score = find_best_lasso(X,y)
#print(lasso)
ground_truth = lasso.coef_
print(ground_truth)
lmbda=lasso.get_params()['alpha']
mu=0.1
intercept=False # check that the best Lasso does not use an intercept.
classification=False
loss='truncated_squared'
penalty='l1'
cut=False
better_init=0
better_radius=10

[-0.  0.  0. ...,  0.  0.  0.]


In [49]:
for n_ellipsoid_steps in [1, 10, 100, 200]: #, 100]: #, 1000]: #, 100000]:
    z, scaling, L, I_k_vec, g , _= experiment_get_ellipsoid(X, y, intercept=intercept, better_init=better_init, better_radius=better_radius, 
                                                         loss=loss, penalty=penalty, lmbda=lmbda, 
                                                         classification=classification, mu=mu, 
                                                         n_ellipsoid_steps=n_ellipsoid_steps)
    z_old, A_old = experiment_get_ellipsoid_old(X, y, intercept=intercept, better_init=better_init, better_radius=better_radius, 
                                                         loss=loss, penalty=penalty, lmbda=lmbda, 
                                                         classification=classification, mu=mu, 
                                                         n_ellipsoid_steps=n_ellipsoid_steps)
    print('L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT', np.linalg.norm(z - ground_truth) / np.linalg.norm(ground_truth))
    print('L2 ... WITH OLD METHOD', np.linalg.norm(z_old - ground_truth) / np.linalg.norm(ground_truth))
    #print('GROUND TRUTH', ground_truth, 'ELLIPSOID OUTPUT', z)
    
    ranking = rank_dataset_accelerated(X, y, z, scaling, L, I_k_vec, g, lmbda, mu, classification, 
    loss, penalty, intercept, cut)
    #print('RANKING', ranking, np.argsort(ranking))
    M = L.dot(np.dot(np.diag(I_k_vec), L.T))
    A = scaling * np.identity(X.shape[1]) - M
    radius, _ = np.linalg.eigh(A)
    #print(radius)
    #print(A)
    #radius = np.linalg.eig(A)[0]
    #print(radius)
    #radius = np.max(radius)
    #print('RADIUS', np.sort(radius))
    #print('SCALING', scaling)
    #print('I_k_vec', I_k_vec)
    #radius_ = np.linalg.eig(M)[0]
    #print('RADIUS_', np.sort(radius_))
    #volume = compute_ellipsoid_volume(radius)
    #print('VOLUME', volume)

Time to compute z, A and g: 0.026360511779785156
L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT 1.00000622587
L2 ... WITH OLD METHOD 1.00000623114
Time to rank the entire dataset: 0.018772363662719727
Time to compute z, A and g: 0.1354508399963379
L2 DISTANCE BETWEEN GROUND TRUTH AND ELLIPSOID OUTPUT 1.00006544769
L2 ... WITH OLD METHOD 1.00006545163
Time to rank the entire dataset: 0.0388791561126709
Time to compute z, A and g: 1.2118866443634033


KeyboardInterrupt: 

# plutôt que le volume, printer le test sur l'ellipsoide ?